# A

In [1]:
H, A = 10, 4

In [2]:
print(-(-H // A))

3


# B

In [4]:
H, N = 10, 3
A = [4, 5, 6]

In [5]:
print("Yes" if H <= sum(A) else "No")

Yes


# C

In [13]:
N, K = 3, 1
H = [4, 1, 5]

print(5)

5


In [15]:
N, K = 8, 9
H = [7, 9, 3, 2, 3, 8, 4, 6]


5


In [16]:
H.sort(reverse=True)
print(sum(H[K:]))

0


# D

In [36]:
H = 2

print(3)

3


In [41]:
H = 4

print(7)

7


In [43]:
H = 1000000000000

print(1099511627775)

1099511627775


In [45]:
x = H
cnt = 1
ans = 0
while x > 1:
    x //= 2
    ans += cnt
    cnt *= 2
print(ans + cnt)

1099511627775


# E

In [73]:
H, N = 9, 3
X = [[8, 3], [4, 2], [2, 1]]

print(4)

4


In [83]:
H, N = 9, 3
X = [[9, 12], [8, 2], [2, 1]]

print(3)

3


In [85]:
H, N = 9999, 10
X = [[540, 7550], [691, 9680], [700, 9790], [510, 7150], [415, 5818],
     [551, 7712], [587, 8227], [619, 8671], [588, 8228], [176, 2461]]

print(139815)

139815


In [87]:
dp = [[10 ** 9] * (H + 1) for _ in range(N + 1)]
dp[0][0] = 0
X.sort(key=lambda x: -x[-1])

for i, (a, b) in enumerate(X):
    for j in range(H + 1):
        if j == 0:
            dp[i + 1][j] = 0
        if j + a < H:
            dp[i + 1][j + a] = min(dp[i][j + a], dp[i][j] + b, dp[i + 1][j] + b)
        else:
            dp[i + 1][-1] = min(dp[i][-1], dp[i + 1][-1], dp[i][j] + b, dp[i + 1][j] + b)            
            
print(dp[-1][-1])

139815


# F

In [1]:
N, D, A = 3, 3, 2
XH = [[1, 2], [5, 4], [9, 2]]

print(2)

2


In [42]:
N, D, A = 9, 4, 1
XH = [[6, 2], [7, 3], [8, 4], [9, 5],
      [1, 5], [2, 4], [3, 3], [4, 2], [5, 1]]

print(5)

5


#### 自分の解法

In [39]:
class BIT:
    def __init__(self, n=None):
        # 1-index
        if n is None:
            n = (1 << 20) - 1
        self.n = n
        self.dat_a = [0] * (2 * self.n - 1)
        self.dat_b = [0] * (2 * self.n - 1)

    def add(self, a, b, x, k=0, l=0, r=None):
        # Add x to [a, b)
        if r is None:
            r = self.n

        if a <= l and r <= b:
            self.dat_a[k] += x
        elif l < b and a < r:
            self.dat_b[k] += (min(b, r) - max(a, l)) * x
            self.add(a, b, x, k * 2 + 1, l, (l + r) // 2)
            self.add(a, b, x, k * 2 + 2, (l + r) // 2, r)
            
    def query(self, a, b, k=0, l=0, r=None):
        # Retrun sum of [a, b)
        if r is None:
            r = self.n
        
        if b <= l or r <= a:
            return 0
        
        if a <= l and r <= b:
            return self.dat_a[k] * (r - l) + self.dat_b[k]
        
        res = (min(b, r) - max(a, l)) * self.dat_a[k]
        res += self.query(a, b, k * 2 + 1, l, (l + r) // 2)
        res += self.query(a, b, k * 2 + 2, (l + r) // 2, r)
        
        return res

In [43]:
ans = 0
bit = BIT()
X.sort(key=lambda x: x[0])
for x, h in X:
    res = h - bit.query(x, x + 1)
    if res <= 0:
        continue
        
    cnt = -(-res // A)
    bit.add(x, x + 2 * D + 1, cnt * A)
    ans += cnt
    
print(ans)

5


#### 模範解答

In [1]:
N, D, A = 3, 3, 2
XH = [[1, 2], [5, 4], [9, 2]]

print(2)

2


In [2]:
import bisect

XH.sort()
X, H = zip(*XH)

K = [-(-h // A) for h in H]
L = [0] * (N + 2)

ans = 0
for i in range(N):
    L[i + 1] += L[i]
    s = max(K[i] - L[i + 1], 0)
    ans += s
    L[i + 1] += s
    L[bisect.bisect_right(X, X[i] + 2 * D) + 1] -= s

print(ans)

2


#### BITの解法

解答：http://drken1215.hatenablog.com/entry/2020/01/26/234000

In [13]:
class BIT:
    def __init__(self, n):
        # 1-index
        self.n = n
        self.dat = [[0] * self.n for _ in range(2)]
        
    def _add_body(self, p, i, x):
        while i < self.n:
            self.dat[p][i] += x
            i += i & -i

    def add(self, a, b, x):
        # Add x to [a, b)
        self._add_body(0, a, x * -(a - 1))
        self._add_body(1, a, x)
        self._add_body(0, b, x * (b - 1))
        self._add_body(1, b, x * -1)

    def _query_body(self, p, i):
        res = 0
        while i > 0:
            res += self.dat[p][i]
            i -= i & -i
        return res
        
    def query(self, a, b):
        # Return sum of [a, b)
        res = (self._query_body(0, b - 1)
               + self._query_body(1, b - 1) * (b - 1)
               - self._query_body(0, a - 1)
               - self._query_body(1, a - 1) * (a - 1))
        return res

    def __repr__(self):
        return ", ".join([str(self.query(i, i + 1)) for i in range(self.n)])

In [14]:
N, D, A = 3, 3, 2
XH = [[1, 2], [5, 4], [9, 2]]

print(2)

2


In [17]:
import bisect

X, H = zip(*sorted(XH))
bit = BIT(N + 10)

# 初期化
for i, h in enumerate(H):
    bit.add(i + 1, i + 2, h)
    
# 処理
res = 0
for i in range(N):
    cur = bit.query(i + 1, i + 2)
    if cur <= 0:
        continue

    need = -(-cur // A)
    idx = bisect.bisect_right(X, X[i] + 2 * D) + 1
    bit.add(i + 1, idx + 1, -need * A)
    res += need
    
print(res)

2
